In [268]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [52]:
"""
kaggle competition : Spaceship Titanic (v1.0 train and test)
"""
url = "/kaggle/input/spaceship-titanic/train.csv"
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [53]:
import torch
import pandas as pd
import numpy as np

In [54]:
torch.cuda.is_available()
print(torch.cuda.is_available())
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
print(device)

False
cpu


In [55]:
data = pd.read_csv(url)
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [56]:
#one-hot
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [57]:
for item in data.columns:
    data[item].fillna(0, inplace = True)

print(data.VRDeck.iloc[1:30])

1      44.0
2      49.0
3     193.0
4       2.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11    124.0
12    113.0
13     24.0
14      7.0
15      0.0
16      0.0
17    637.0
18      0.0
19      0.0
20    679.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
Name: VRDeck, dtype: float64


In [58]:
HoemPlanet1 = pd.get_dummies(data.HomePlanet)
CryoSleep1 =  pd.get_dummies(data.CryoSleep)
Destination1 = pd.get_dummies(data.Destination)
VIP1 = pd.get_dummies(data.VIP)


In [59]:
#delete and join
for item in ["HomePlanet","CryoSleep","Destination","VIP"]:
    del data[item]

for item in [HoemPlanet1,CryoSleep1,Destination1,VIP1]:
    data = data.join(item,rsuffix='_r')



/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:9138: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0'} in the result is deprecated and will raise a MergeError in a future version.
  sort=sort,


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    8693 non-null   object 
 1   Cabin          8693 non-null   object 
 2   Age            8693 non-null   float64
 3   RoomService    8693 non-null   float64
 4   FoodCourt      8693 non-null   float64
 5   ShoppingMall   8693 non-null   float64
 6   Spa            8693 non-null   float64
 7   VRDeck         8693 non-null   float64
 8   Name           8693 non-null   object 
 9   Transported    8693 non-null   bool   
 10  0              8693 non-null   uint8  
 11  Earth          8693 non-null   uint8  
 12  Europa         8693 non-null   uint8  
 13  Mars           8693 non-null   uint8  
 14  False_r        8693 non-null   uint8  
 15  True           8693 non-null   uint8  
 16  0              8693 non-null   uint8  
 17  55 Cancri e    8693 non-null   uint8  
 18  PSO J318

In [61]:
del data['Cabin']

In [62]:
for i in range(len(data.PassengerId)):
    data['PassengerId'][i] = int(data['PassengerId'][i].replace("_",""))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
data['PassengerId'] = data['PassengerId'].astype(int)

In [64]:
# for i in range(len(data.Name)):
#     data['Name'][i] = len(str(data['Name'][i]))



In [65]:
data['Name'] = data['Name'].str.len()

In [66]:
data['Name']

0       15.0
1       12.0
2       13.0
3       12.0
4       17.0
        ... 
8688    17.0
8689    15.0
8690    12.0
8691    16.0
8692    16.0
Name: Name, Length: 8693, dtype: float64

In [67]:
data.Transported = data.Transported.replace([False,True],[0,1])

In [68]:
data.Transported

0       0
1       1
2       0
3       0
4       1
       ..
8688    0
8689    0
8690    1
8691    0
8692    1
Name: Transported, Length: 8693, dtype: int64

In [69]:
Y_data =  data.Transported.values.reshape(-1,1)
del data['Transported']
X_data =  data.values
norms = np.linalg.norm(X_data, axis=1)
norms = norms.reshape(-1,1)
X_data = X_data / norms


In [70]:
X_data.shape

(8693, 20)

In [71]:
X = torch.from_numpy(X_data).type(torch.float32)
Y = torch.from_numpy(Y_data).type(torch.float32)


In [72]:
X_data.shape

(8693, 20)

In [73]:
X = torch.nan_to_num(X)
Y = torch.nan_to_num(Y)

In [74]:
Y_data.shape
from torch import nn

In [76]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(20,12)
        self.linear2 = nn.Linear(12, 6)
        self.linear3 = nn.Linear(6, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    def forward(self,input):
        x = self.linear1(input)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.sigmoid(x)
        return x

In [77]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

"""
设置训练相关参数
"""
model = Model()
epochs = 200
batch_size = 64
loss_fun = torch.nn.BCELoss()
lr = 0.001
opt = torch.optim.Adam(model.parameters(),lr=lr)

#平均准确率函数
def accuracy(y_pred,y_true):
    y_pred = (y_pred > 0.6).type(torch.int32)
    acc = (y_pred == y_true).type(torch.float32).mean()
    return acc

In [78]:
dataset = TensorDataset(X,Y)

In [79]:
dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [80]:
lossa = []
acca = []
epocha = []
for epoch in range(epochs):
    for x,y in dataloader:
        #x = x.to(device)
        #y = y.to(device)
        y_pred = model(x)
        loss = loss_fun(y_pred,y)
        opt.zero_grad()
        loss.backward()
        opt.step()
    with torch.no_grad():
        #训练平均准确率和损失
        epoch_accuracy = accuracy(model(X),Y)
        epoch_loss = loss_fun(model(X),Y).data
        lossa.append(epoch_loss)
        acca.append(epoch_loss)
        epocha.append(epoch)
        print("epoch:",epoch,
              "loss:",epoch_loss,"accuracy:",epoch_accuracy,
              )

epoch: 0 loss: tensor(0.6931) accuracy: tensor(0.4964)
epoch: 1 loss: tensor(0.6929) accuracy: tensor(0.4964)
epoch: 2 loss: tensor(0.6927) accuracy: tensor(0.4964)
epoch: 3 loss: tensor(0.6925) accuracy: tensor(0.4964)
epoch: 4 loss: tensor(0.6922) accuracy: tensor(0.4964)
epoch: 5 loss: tensor(0.6920) accuracy: tensor(0.4964)
epoch: 6 loss: tensor(0.6915) accuracy: tensor(0.4964)
epoch: 7 loss: tensor(0.6911) accuracy: tensor(0.4964)
epoch: 8 loss: tensor(0.6906) accuracy: tensor(0.4965)
epoch: 9 loss: tensor(0.6898) accuracy: tensor(0.4966)
epoch: 10 loss: tensor(0.6891) accuracy: tensor(0.4968)
epoch: 11 loss: tensor(0.6883) accuracy: tensor(0.4967)
epoch: 12 loss: tensor(0.6873) accuracy: tensor(0.4971)
epoch: 13 loss: tensor(0.6861) accuracy: tensor(0.4974)
epoch: 14 loss: tensor(0.6849) accuracy: tensor(0.4979)
epoch: 15 loss: tensor(0.6835) accuracy: tensor(0.4982)
epoch: 16 loss: tensor(0.6822) accuracy: tensor(0.4997)
epoch: 17 loss: tensor(0.6808) accuracy: tensor(0.4987)
ep

In [82]:
%matplotlib inline
import matplotlib.pyplot as plt # 导入绘图包

In [83]:
# plt.figure(20,2)
# plt.subplot(2,1,1)
# plt.plot(epocha,lossa)
# plt.subplot(2,1,2)
# plt.plot(epocha,acca)

In [84]:
url2 = "/kaggle/input/spaceship-titanic/test.csv"

In [85]:
data2 =  pd.read_csv(url2)

In [86]:
data2

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [87]:
for item in data2.columns:
    data2[item].fillna(0, inplace = True)
    
HoemPlanet2 = pd.get_dummies(data2.HomePlanet)
CryoSleep2 =  pd.get_dummies(data2.CryoSleep)
Destination2 = pd.get_dummies(data2.Destination)
VIP2 = pd.get_dummies(data2.VIP)




In [88]:
del data2['Cabin']
#delete and join
for item in ["HomePlanet","CryoSleep","Destination","VIP"]:
    del data2[item]



In [89]:
for item in [HoemPlanet2,CryoSleep2,Destination2,VIP2]:
    data2 = data2.join(item,rsuffix='_r')



/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:9138: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0'} in the result is deprecated and will raise a MergeError in a future version.
  sort=sort,


In [90]:
for i in range(len(data2.PassengerId)):
    data2['PassengerId'][i] = int(data2['PassengerId'][i].replace("_",""))




/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [91]:
data2['PassengerId'] = data2['PassengerId'].astype(int)
data2['Name'] = data2['Name'].str.len()

# for item in data2.columns:
#     data2[item].fillna(0, inplace = True)

    
    
X_data2 =  data2.values

In [92]:
X_data2.shape

(4277, 20)

In [93]:
X_data2 = X_data2.astype(float)
X_data2
norms = np.linalg.norm(X_data2, axis=1)
norms = norms.reshape(-1,1)
X_data2 = X_data2 / norms



In [97]:
X_data.shape

(8693, 20)

In [99]:
X2 = torch.from_numpy(X_data2).type(torch.float32)

In [103]:
for i in (model(X2) > 0.5):
    print(i)
    

tensor([True])
tensor([False])
tensor([True])
tensor([True])
tensor([True])
tensor([True])
tensor([True])
tensor([True])
tensor([True])
tensor([True])
tensor([False])
tensor([False])
tensor([True])
tensor([True])
tensor([False])
tensor([False])
tensor([False])
tensor([True])
tensor([True])
tensor([False])
tensor([True])
tensor([False])
tensor([True])
tensor([True])
tensor([True])
tensor([False])
tensor([True])
tensor([True])
tensor([True])
tensor([True])
tensor([True])
tensor([False])
tensor([True])
tensor([True])
tensor([False])
tensor([False])
tensor([True])
tensor([True])
tensor([True])
tensor([False])
tensor([True])
tensor([False])
tensor([True])
tensor([True])
tensor([False])
tensor([True])
tensor([False])
tensor([False])
tensor([True])
tensor([False])
tensor([True])
tensor([True])
tensor([False])
tensor([True])
tensor([False])
tensor([True])
tensor([False])
tensor([True])
tensor([True])
tensor([False])
tensor([True])
tensor([False])
tensor([False])
tensor([True])
tensor([True])
t